In [ ]:
%load_ext autoreload
%autoreload 2
from lib import load, timing

YEAR = 2024
DAY = 15
TESTDATA = [
    None,
    '##########\n#..O..O.O#\n#......O.#\n#.OO..O.O#\n#..O@..O.#\n#O#..O...#\n#O..O..O.#\n#.OO.O.OO#\n#....O...#\n##########\n\n<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^\nvvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v\n><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<\n<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^\n^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><\n^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^\n>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^\n<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>\n^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>\nv^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^\n'
][0]
TEST = TESTDATA is None

In [ ]:
from lib import CharGrid

@timing
def prepare_data():
    data = load(YEAR, DAY, split_lines=True, test=TESTDATA if TEST else None)

    input_break = next(i for i, row in enumerate(data['split']) if row == '')
    
    grid = CharGrid(data['split'][:input_break])
    initial = grid.find('@')
    moves = ''.join(data['split'][input_break:])

    return grid, initial, moves

In [ ]:
# Level 1
def next_pos(grid, pos, direction):
    if   direction == '^':  return grid.up(pos)
    elif direction == 'v':  return grid.down(pos)
    elif direction == '<':  return grid.left(pos)
    elif direction == '>':  return grid.right(pos)


def move1(grid, pos, direction):
    # build push chain
    towards = [next_pos(grid, pos, direction)]
    while grid[towards[-1]] == 'O':
        towards.append(next_pos(grid, towards[-1], direction))

    # end of chain is a wall: do nothing
    if grid[towards[-1]] == '#': 
        return pos

    # push the chain: it's all boxes, so only the beginning and end of the chain need to be changed
    grid[pos] = '.'
    grid[towards[0]] = '@'
    if len(towards) > 1:
        grid[towards[-1]] = 'O'
    return towards[0]


def score_boxes(grid):
    for p in grid.walk():
        if grid[p] in 'O[':
            x, y = p
            yield x + 100 * y


@timing
def level1(grid, initial, moves):
    pos = initial
    for steps, direction in enumerate(moves, start = 1):
        old_pos = pos
        pos = move1(grid, pos, direction)
        
    return sum(score_boxes(grid))


grid, initial, moves = prepare_data()
print(level1(grid, initial, moves))

In [ ]:
# Level 2
from itertools import pairwise

def move2_horizontal(grid, pos, direction):
    # build push chain
    towards = [pos, next_pos(grid, pos, direction)]
    while grid[towards[-1]] in '[]':
        towards.append(next_pos(grid, towards[-1], direction))

    # end of chain is a wall: do nothing
    if grid[towards[-1]] == '#': 
        return pos

    # push the chain:
    for pos1, pos2 in pairwise(towards[::-1]):
        grid[pos1] = grid[pos2]
    grid[pos] = '.'
    return towards[1]


def move2_vertical(grid, pos, direction):
    # build push tree
    towards = {pos[0]: [next_pos(grid, pos, direction)]}
    boxes = True
    while boxes:
        # complete boxes
        add = []
        for new_pos in [tw[-1] for tw in towards.values()]:
            if grid[new_pos] == '#': # reached a wall, no move possible
                return pos
            elif grid[new_pos] == '[': # left half of a box - need to move right half too
                add.append(grid.right(new_pos))
            elif grid[new_pos] == ']': # right half of a box - need to move left half too
                add.append(grid.left(new_pos))
        boxes = len(add) != 0
        
        # update push tree
        for new_pos in add:
            x = new_pos[0]
            if not x in towards:
                towards[x] = [new_pos]
            elif towards[x][-1] != new_pos:
                towards[x].append(new_pos)
        
        # build next row:
        for new_pos in [tw[-1] for tw in towards.values()]:
            if grid[new_pos] in '[]': # reached a box, need to check next row
                towards[new_pos[0]].append(next_pos(grid, new_pos, direction))

    # push the tree:
    for towards_x in towards.values():
        for pos1, pos2 in pairwise(towards_x[::-1]):
            if abs(pos1[0] - pos2[0]) == 0:
                grid[pos1] = grid[pos2]
                grid[pos2] = '.'

    grid[pos] = '.'
    new_pos = towards[pos[0]][0]
    grid[new_pos] = '@'
    return new_pos


def move2(grid, pos, direction):
    # build push chain
    if direction in '<>':
        return move2_horizontal(grid, pos, direction)
    else:
        return move2_vertical(grid, pos, direction)


@timing
def level2(grid, initial, moves):
    scaler = {'#': '##', '.': '..', 'O': '[]', '@': '@.'}
    grid = CharGrid([''.join([scaler[c] for c in row]) for row in grid.grid])
    initial = (initial[0] * 2, initial[1])

    pos = initial
    for steps, direction in enumerate(moves, start = 1):
        old_pos = pos
        pos = move2(grid, pos, direction)
        
    return sum(score_boxes(grid))


grid, initial, moves = prepare_data()
print(level2(grid, initial, moves))